# SLURM with environment variables

Let's say you want to pass the number of nodes, e.g., as a command
line input,  `-N $1` for the number of nodes. 
It's not possible to do this by setting variables within the 
slurm script before the `#SBATCH` stuff is called and using 
command line arguments for the script. So,  

```bash
#!/bin/bash

# Get environment variables.
DATADIR=$1
NODES=$(($3))
TASKS=$(($4))

#SBATCH -t 00:00:20
#SBATCH --export=RUNDIR=$RUNDIR,NODES=$NODES,NPAR=$NPAR,TASKS=$TASKS
#SBATCH --job-name=job_array_%a
#SBATCH --output=$RUNDIR/output/job_%a.out
#SBATCH --error=$RUNDIR/output/job_%a.err
#SBATCH -N $NODES
#SBATCH -n $TASKS
#SBATCH --array=0-1

# Create String array as a function of wanted parameters
STRINGS= (0 1)

# Change to CMT simulation directory
cd "$RUNDIR/${STRINGS[${SLURM_ARRAY_TASK_ID}]}"

# Execute script
srun python demo.py input/param_${STRINGS[${SLURM_ARRAY_TASK_ID}]}.txt

```
and then

```bash
$ sbatch run.sh
```

wouldn't work.

# The workaround

A work around however is to set the environment variables beforehand
and export them into `SBATCH` script the following way, if needed, and 
create an overloaded SBATCH function call.

1. Set the environment variables:

```bash
DATADIR=<your_stuff>
NODES=1
TASKS=1
```


2. Create a new script folowing the previous one.
    
```bash
#!/bin/bash

#SBATCH -t 00:00:20
#SBATCH --job-name=job_array_%a
#SBATCH --output=./output/job_%a.out
#SBATCH --error=./output/job_%a.err

# Create String array as a function of wanted parameters
STRINGS= (0 1)

# Change to CMT simulation directory
cd $RUNDIR

# Execute script
srun /bin/python demo.py input/param_${STRINGS[${SLURM_ARRAY_TASK_ID}]}.txt
```


3. Run the script by importing stuff
    
```bash
sbatch -N $NODES -n $TASKS --array=0-1 --export=DATADIR=$DATADIR,NODES=$NODES,TASKS=$TASKS run.sh
```

4. Now, make the above command a script where you take the number of Nodes and tasks as a
command line argument:

```bash
#!/bin/bash

# Get command line arguments
NODES=$1
TASKS=$2
RUNDIR=$3
SCRIPT=$4

# Run batch script          # array sets the number of jobs.
sbatch -N $NODES -n $TASKS --array=0-1 --export=RUNDIR=$DATADIR,NODES=$NODES,TASKS=$TASKS $SCRIPT 
```
